In [2]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import importlib
from yapf.yapflib.yapf_api import FormatCode
import GPUtil

# Project imports
import utils

In [3]:
main_path = '/home/rfradkin/xom/'
raw_tess_path = '/scratch/data/tess/lcur/spoc/raws/'
tess_metad_path = '/data/scratch/data/tess/meta/'
xom_data_path = '/data/scratch/data/moon/'

### GPU Setup

In [10]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 | 50% | 19% |
|  1 | 52% | 12% |
|  2 | 54% | 12% |
|  3 | 55% | 12% |


In [6]:
# All GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
mirror_strat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


### Data Preprocessing

In [ ]:
cuts_type = 'padde_cuts-detre-'
model_file = f'padde_cuts-1_1-parti-1637036581.5612013.pkl-88.491.h5'
cutof = 0.5
candi = []

In [ ]:
numbe_sampl = -1
masking = layers.Masking(mask_value=0., input_shape=(1900, 1))
model = tf.keras.models.load_model(
    f'{xom_data_path}plane_moon_model/{model_file}')

for secto in range(1, 27):

    datas = f'{cuts_type}{secto}_{secto + 1}.pkl'
    with open(f'{xom_data_path}padde_cuts/{datas}', 'rb') as f:
        raw_x_data = pkl.load(f)

    # Create a shuffled sample of the data
    rando_shuff_x_data = raw_x_data[utils.retur_rando_sampl(
        numbe_sampl, len(raw_x_data))]
    np.random.shuffle(rando_shuff_x_data)

    # Normalize the data from 0 to 1
    norma_full_x_data = np.copy(rando_shuff_x_data)
    x_data = np.copy(rando_shuff_x_data[:, :-1, 1].astype(float))

    for i in range(len(x_data)):
        chang_slots = np.where(x_data[i] != 0)[0]
        x_data[i, chang_slots] = utils.norma_data(x_data[i, chang_slots])
        norma_full_x_data[i, :-1, 1] = x_data[i]

    x_data = np.expand_dims(x_data, 2)

    # Create the corresponding y dataset
    y_data = np.zeros(len(norma_full_x_data))
    for i in range(len(norma_full_x_data)):
        y_data[i] = norma_full_x_data[i, -1, 1][
            'plane_moon_cut_injec'] or norma_full_x_data[i, -1,
                                                         1]['plane_cut_injec']

    x_test_np = np.array(x_data)
    full_x_test = norma_full_x_data
    y_test_np = y_data

    # Mask the data
    x_test_unmas = tf.convert_to_tensor(x_test_np)
    y_test = tf.convert_to_tensor(y_test_np)
    x_test = masking(x_test_unmas)

    # Make predictions
    predi = model.predict(x_test).squeeze()
    binar_predi = predi > cutof

    for i in range(len(predi)):
        if binar_predi:
            candi.append([
                predi[i], secto, full_x_test[i, -1, 1]['tic_id'],
                full_x_test[i]
            ])

print(f'{len(candi)} candidates identified.')

### Formatting